# prospect + specutils

Project: Convert DESI, SDSS & eBOSS spectra to specutils container objects, and modify prospect to accept those objects.

In [8]:
import os
from astropy.table import Table
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
from prospect.specutils import read_spectra, read_spPlate

## DESI spectra file

DESI spectra are stored by "channel" ("channel", "band" and "spectrograph arm" are used interchangably).  There are 10 spectrographs, each with three arms, called 'b', 'r' & 'z'.  In a DESI spectra file, all arms are grouped together, with a common wavelength solution for each arm, but the solutions do not overlap or have the same shape.  Thus we can't use a Spectrum1D or SpectrumCollection object, but we can use a SpectrumList containing three Spectrum1D objects.

In [2]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'datachallenge', 'reference_runs', '19.12', 'spectro', 'redux')
os.environ['SPECPROD'] = 'mini'
nside = '64'
pixnum = '5305'
pixgroup = pixnum[0:2]
desi_spectra = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                            f'spectra-{nside}', pixgroup, pixnum, f'spectra-{nside}-{pixnum}.fits')
desi_redshifts = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                              f'spectra-{nside}', pixgroup, pixnum, f'zbest-{nside}-{pixnum}.fits')
print(desi_spectra)
print(desi_redshifts)

/Users/weaver/Documents/Data/desi/datachallenge/reference_runs/19.12/spectro/redux/mini/spectra-64/53/5305/spectra-64-5305.fits
/Users/weaver/Documents/Data/desi/datachallenge/reference_runs/19.12/spectro/redux/mini/spectra-64/53/5305/zbest-64-5305.fits


## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [3]:
os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr16', 'sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/26/2955/spPlate-2955-54562.fits
/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/26/2955/spZbest-2955-54562.fits


In [4]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/v5_13_0/9599/spPlate-9599-58131.fits
/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/v5_13_0/9599/v5_13_0/spZbest-9599-58131.fits


In [5]:
for f in (desi_spectra, desi_redshifts, sdss_spectra, sdss_redshifts, eboss_spectra, eboss_redshifts):
    assert os.path.exists(f)

In [6]:
desi = read_spectra(desi_spectra)
desi_z = Table.read(desi_redshifts, 'ZBEST')
desi

[<Spectrum1D(flux=<Quantity [[-0.84117043, -0.68834341, -1.40195751, ..., -0.59830803,
              5.19148731,  0.35107699],
            [-0.56885755,  1.45553529,  6.3197751 , ...,  0.25085509,
              3.68603396, -2.282166  ],
            [-2.48546124,  3.44139385, -6.46033764, ..., -2.77618814,
             -0.15636389, -2.45203424],
            ...,
            [ 3.57696438,  6.76710939,  3.24615264, ..., -6.3416853 ,
             -3.2689023 , -5.22456694],
            [ 0.77733421, 12.95565128,  4.07372427, ..., -5.95146942,
             10.68841171,  2.36848235],
            [-2.92037797,  3.95976591, 12.81771183, ..., 32.75897217,
             37.15966797, 17.1248188 ]] 1e-17 erg / (Angstrom cm2 s)>, spectral_axis=<SpectralCoord [3569.3, 3570.1, 3570.9, ..., 5946.9, 5947.7, 5948.5] Angstrom, 
 	radial_velocity=0.0 km / s, 
 	redshift=0.0, 
 	doppler_rest=0.0 Angstrom, 
 	doppler_convention=None, 
 	observer=None, 
 	target=None>, uncertainty=InverseVariance([[0.12316658,

In [9]:
sdss = read_spPlate(sdss_spectra)
sdss_z = Table.read(sdss_redshifts, 1)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
a string value was expected. [astropy.wcs.wcs]
a string value was expected.


In [10]:
sdss

<Spectrum1D(flux=<Quantity [[11.0880165 , 11.084556  , 11.081101  , ..., 11.787075  ,
            11.787052  , 11.787028  ],
           [ 4.371129  ,  4.36966   ,  4.3681927 , ...,  2.2523484 ,
             2.2523346 ,  2.2523208 ],
           [ 0.85558206,  0.8553027 ,  0.8550237 , ...,  8.030891  ,
             8.030855  ,  8.03082   ],
           ...,
           [18.067533  , 18.060213  , 18.052906  , ..., 11.21297   ,
            11.213058  , 11.213148  ],
           [20.821095  , 20.811958  , 20.802837  , ...,  7.5393324 ,
             7.53935   ,  7.539367  ],
           [ 7.2744026 ,  7.271167  ,  7.267936  , ...,  3.3903959 ,
             3.3904004 ,  3.3904047 ]] 1e-17 erg / (Angstrom cm2 s)>, spectral_axis=<SpectralCoord [3802.76948244, 3803.64520329, 3804.5211258 , ...,
                9217.22098659, 9219.34357452, 9221.46665124] Angstrom, 
	radial_velocity=0.0 km / s, 
	redshift=0.0, 
	doppler_rest=0.0 Angstrom, 
	doppler_convention=None, 
	observer=None, 
	target=None>, un

In [11]:
sdss_z

PLATE,TILE,MJD,FIBERID,OBJID [5],OBJTYPE,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,DOF,RCHI2DIFF,TFILE,TCOLUMN [10],NPOLY,THETA [10],"THETA_COVAR [10,10]",VDISP,VDISP_ERR,VDISPZ,VDISPZ_ERR,VDISPCHI2,VDISPNPIX,VDISPDOF,WAVEMIN,WAVEMAX,WCOVERAGE,ZWARNING,SN_MEDIAN,CHI68P,FRACNSIGMA [10],FRACNSIGHI [10],FRACNSIGLO [10],SPECTROFLUX [5],SPECTROFLUX_IVAR [5],SPECTROSYNFLUX [5],SPECTROSYNFLUX_IVAR [5],SPECTROSKYFLUX [5],ANYANDMASK,ANYORMASK,SPEC1_G,SPEC1_R,SPEC1_I,SPEC2_G,SPEC2_R,SPEC2_I,ELODIE_FILENAME,ELODIE_OBJECT,ELODIE_SPTYPE,ELODIE_BV,ELODIE_TEFF,ELODIE_LOGG,ELODIE_FEH,ELODIE_Z,ELODIE_Z_ERR,ELODIE_Z_MODELERR,ELODIE_RCHI2,ELODIE_DOF
int32,int32,int32,int32,int32,bytes19,float64,float64,bytes6,bytes21,float32,float32,float32,int32,float32,bytes24,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,float32,float32,float32,float32,float32,float32,bytes25,bytes21,bytes9,float32,float32,float32,float32,float32,float32,float32,float32,int32
2955,1929,54562,1,2327 .. 410,GALAXY,236.31232,0.85039527,GALAXY,,0.21037471,4.4343204e-05,1.2863714,3719,0.37695348,spEigenGal-53724.fits,0 .. -1,3,0.0014049356 .. 0.0,1.7609564e-09 .. 0.0,170.17068,14.28543,0.0,0.0,2389.8547,2168.0,2117,3822.082,9191.788,0.3726,0,8.955571,1.1095116,0.37198067 .. 0.0,0.18518518 .. 0.0,0.18679549 .. 0.0,5.528316 .. 78.13351,1.5567644 .. 0.68778366,2.5305223 .. 77.650925,2.1609764 .. 0.8614444,5.73486 .. 86.28137,230752256,266272768,20.601,21.1441,20.8762,24.9549,26.7781,23.0632,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2955,1929,54562,2,2327 .. 351,GALAXY,236.70843,0.90606157,GALAXY,STARFORMING,0.045228824,9.098427e-06,1.4144219,3726,0.75410926,spEigenGal-53724.fits,0 .. -1,3,0.001198203 .. 0.0,3.053975e-09 .. 0.0,42.783955,29.819418,0.0,0.0,2590.0378,2168.0,2117,3822.082,9191.788,0.3733,0,6.4640083,1.1339878,0.38092688 .. 0.0,0.19126707 .. 0.0,0.18965979 .. 0.0,6.2608438 .. 34.75663,1.5567644 .. 0.68778366,4.4604278 .. 35.09296,2.1609764 .. 0.8614444,6.5782337 .. 83.10503,230752256,266272768,20.601,21.1441,20.8762,24.9549,26.7781,23.0632,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2955,1929,54562,3,2327 .. 544,GALAXY,236.58816,0.91476191,GALAXY,,0.26219922,6.426468e-05,1.1840944,3735,0.16153336,spEigenGal-53724.fits,0 .. -1,3,0.000999022 .. 0.0,2.6019062e-09 .. 0.0,174.73615,19.371538,0.0,0.0,2368.895,2168.0,2117,3821.2021,9191.788,0.3742,0,5.264221,1.0683659,0.35088187 .. 0.0,0.17691074 .. 0.0,0.17397113 .. 0.0,1.6958325 .. 63.58714,1.5567644 .. 0.68778366,0.82249457 .. 63.130188,2.1609764 .. 0.8614444,5.863977 .. 89.36571,231800836,266272772,20.601,21.1441,20.8762,24.9549,26.7781,23.0632,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2955,1929,54562,4,2327 .. 192,GALAXY,236.08242,0.8743279,GALAXY,,0.12016291,3.6281606e-05,1.1879362,3725,0.40690196,spEigenGal-53724.fits,0 .. -1,3,0.0016338797 .. 0.0,2.3526507e-09 .. 0.0,161.72289,11.979559,0.0,0.0,2127.9797,2168.0,2117,3821.2021,9189.672,0.3732,0,11.293284,1.0705984,0.35744908 .. 0.0,0.17524116 .. 0.0,0.18220793 .. 0.0,4.2502537 .. 93.75019,1.5567644 .. 0.68778366,4.5496674 .. 94.6758,2.1609764 .. 0.8614444,6.7003837 .. 85.79785,230752256,266272768,20.601,21.1441,20.8762,24.9549,26.7781,23.0632,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2955,1929,54562,5,1458 .. 179,GALAXY,236.15968,0.97761396,GALAXY,STARFORMING,0.085093014,9.0356e-06,1.4179437,3717,1.1750665,spEigenGal-53724.fits,0 .. -1,3,0.00072964723 .. 0.0,2.6174127e-09 .. 0.0,98.21468,16.914415,0.0,0.0,2553.4014,2168.0,2117,3821.2021,9189.672,0.3724,0,9.912004,1.1363326,0.37781954 .. 0.0,0.18931256 .. 0.0,0.18850698 .. 0.0,8.900323 .. 61.221573,1.5567644 .. 0.68778366,7.8341045 .. 62.532814,2.1609764 .. 0.8614444,7.1100903 .. 86.26363,230752256,266272768,20.601,21.1441,20.8762,24.9549,26.7781,23.0632,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2955,1929,54562,6,2327 .. 389,GALAXY,236.29654,0.81158471,GALAXY,,0.20805018,5.3525437e-05,1.195

In [12]:
eboss = read_spPlate(eboss_spectra)
eboss_z = Table.read(eboss_redshifts, 1)

a string value was expected. [astropy.wcs.wcs]
a string value was expected.


In [13]:
eboss

<Spectrum1D(flux=<Quantity [[  2.1177    ,   2.116765  ,   2.1158316 , ...,  -1.4393026 ,
             -1.4393069 ,  -1.4393113 ],
           [  8.084128  ,   8.08055   ,   8.076977  , ...,  -1.6968985 ,
             -1.6969033 ,  -1.6969082 ],
           [ 37.020176  ,  37.00411   ,  36.98806   , ...,   3.8933334 ,
              3.8933485 ,   3.893364  ],
           ...,
           [ -5.6615777 ,  -5.659113  ,  -4.6482496 , ...,  -0.17259687,
             -0.17259583,  -0.17259477],
           [ -4.649216  ,  -4.6471095 ,  -0.9829586 , ...,  -9.405368  ,
             -5.7500234 ,  -5.749976  ],
           [-32.441444  , -32.427807  , -19.826235  , ...,   7.553325  ,
              5.2352567 ,   5.2352343 ]] 1e-17 erg / (Angstrom cm2 s)>, spectral_axis=<SpectralCoord [ 3586.7408577 ,  3587.56683039,  3588.39299329, ...,
                10351.42166679, 10353.80544415, 10356.18977045] Angstrom, 
	radial_velocity=0.0 km / s, 
	redshift=0.0, 
	doppler_rest=0.0 Angstrom, 
	doppler_convention

In [14]:
eboss_z

PLATE,TILE,MJD,FIBERID,RUN2D,RUN1D,OBJID [5],OBJTYPE,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,DOF,RCHI2DIFF,TFILE,TCOLUMN [10],NPOLY,THETA [10],"THETA_COVAR [10,10]",VDISP,VDISP_ERR,VDISPZ,VDISPZ_ERR,VDISPCHI2,VDISPNPIX,VDISPDOF,WAVEMIN,WAVEMAX,WCOVERAGE,ZWARNING,SN_MEDIAN [5],SN_MEDIAN_ALL,CHI68P,FRACNSIGMA [10],FRACNSIGHI [10],FRACNSIGLO [10],SPECTROFLUX [5],SPECTROFLUX_IVAR [5],SPECTROSYNFLUX [5],SPECTROSYNFLUX_IVAR [5],SPECTROSKYFLUX [5],ANYANDMASK,ANYORMASK,SPEC1_G,SPEC1_R,SPEC1_I,SPEC2_G,SPEC2_R,SPEC2_I,ELODIE_FILENAME,ELODIE_OBJECT,ELODIE_SPTYPE,ELODIE_BV,ELODIE_TEFF,ELODIE_LOGG,ELODIE_FEH,ELODIE_Z,ELODIE_Z_ERR,ELODIE_Z_MODELERR,ELODIE_RCHI2,ELODIE_DOF,Z_NOQSO,Z_ERR_NOQSO,ZNUM_NOQSO,ZWARNING_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,RCHI2DIFF_NOQSO,VDISP_LNL [35],SPECOBJID
int32,int32,int32,int32,bytes7,bytes7,int32,bytes16,float64,float64,bytes6,bytes19,float32,float32,float32,int32,float32,bytes24,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,float32,float32,float32,float32,float32,float32,bytes16,bytes8,bytes12,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,int32,int32,bytes6,bytes16,float32,float32,uint64
9599,17241,58131,1,v5_13_0,v5_13_0,0 .. 0,GALAXY,134.30633999999998,23.560574,GALAXY,,0.88640094,7.4862786e-05,1.0845052,4415,0.06782746,spEigenGal-56436.fits,0 .. -1,3,-0.0017581133 .. 0.0,2.1188825e-06 .. 0.0,435.1586,175.88791,0.0,0.0,1429.6091,1242.0,1017,3601.6375,10332.371,0.4422,0,0.2606063 .. 1.3497385,0.93927145,0.99712366,0.3186341 .. 0.0,0.16350068 .. 0.0,0.15513343 .. 0.0,3.7662063 .. 5.9771852,2.2663114 .. 0.08043097,0.7791927 .. 5.6669927,6.7186484 .. 1.8029664,9.757312 .. 172.13158,96731136,265224192,5.49435,19.6345,15.2609,6.32216,15.6758,16.1718,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.88640094,7.4862786e-05,1,0,GALAXY,,0.08233416,-734.5371 .. -732.77313,10807513617077129216
9599,17241,58131,2,v5_13_0,v5_13_0,0 .. 0,GALAXY,134.37579,23.661467,GALAXY,,1.3339226,0.00026624298,1.070623,4410,0.0049744844,spEigenGal-56436.fits,0 .. -1,3,0.0064811194 .. 0.0,2.4558062e-06 .. 0.0,640.3693,14.852106,0.0,0.0,336.13528,317.0,189,3601.6375,10332.371,0.4417,4,0.20378432 .. 0.4679302,0.3826797,1.004011,0.32171157 .. 0.0,0.16708173 .. 0.0,0.15462984 .. 0.0,-0.067608245 .. 2.0342577,2.2663114 .. 0.08043097,0.2092362 .. 2.2073357,6.7186484 .. 1.8029664,9.159222 .. 172.65573,130023424,265224192,5.49435,19.6345,15.2609,6.32216,15.6758,16.1718,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.3339226,0.00026624298,1,4,GALAXY,,0.0049744844,-186.14551 .. -186.13846,10807513891955036160
9599,17241,58131,3,v5_13_0,v5_13_0,4576 .. 23,NA,134.32067,23.688328,QSO,BROADLINE,1.763546,0.0003326265,1.251095,4426,1.5217528,spEigenQSO-55732.fits,0 .. -1,3,0.0053143967 .. 0.0,2.0002277e-08 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3601.6375,10332.371,0.4433,0,7.3063717 .. 11.766265,16.69086,1.0579199,0.34829685 .. 0.0,0.18565306 .. 0.0,0.1626438 .. 0.0,50.915344 .. 53.722816,2.2663114 .. 0.08043097,45.645782 .. 54.516926,6.7186484 .. 1.8029664,10.214458 .. 170.73433,97058816,265224192,5.49435,19.6345,15.2609,6.32216,15.6758,16.1718,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.17964284,6.613678e-05,6,4,GALAXY,,0.0026409626,0.0 .. 0.0,10807514166832943104
9599,17241,58131,4,v5_13_0,v5_13_0,0 .. 0,SKY,134.35102,23.701891,QSO,,1.1664659,2.9052196,0.8529913,4413,0.004733205,spEigenQSO-55732.fits,0 .. -1,3,-0.0004680047 .. 0.0,8.280256e-09 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3601.6375,10332.371,0.442,5,0.104121625 .. 0.38119227,0.2799364,0.8960617,0.27194571 .. 0.0,0.13959277 .. 0.0,0.13235295 .. 0.0,0.38857126 .. 2.3111312,2.2663114 .. 0.08043097,0.24660099 .. 1.8778524,6.7186484 .. 1.8029664,8.592404 .. 170.20645,96731136,265224192,5.49435,19.6345,15.2609,6.32216,15.6758,16.1718,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.6151705,-1.0,3,5,GALAXY,,0.0020426512,0.0 .. 0.0,1080